# <center> 721. Accounts Merge </center>


## Problem Description
[Click here](https://leetcode.com/problems/accounts-merge/description/)


## Intuition
<!-- Describe your first thoughts on how to solve this problem. -->
We need to merge the same accounts. Accounts are same if there is a common email address. 
In other words, same nodes (accounts) are connected by a common edge (email address) and form a subgraph (connected component). We need to find the connected components. This can be done using:
1. DFS
2. Union-Find

Unio-Find is fast, we will use the optimized version of the union-find algorithm i.e union by rank and path compression.


## Approach
<!-- Describe your approach to solving the problem. -->
#### **Union-Find Class**
*define a class to implement union-find. There is no need to create a separate class, we can also implement it using nested functions but it's better this way*

**init(total nodes)**
- create a list to store parent of each node/vertex
- create a list to store the rank (height) of each tree. Initially, each node represents a tree and acts as a root node of the tree, so the height is 1 for each tree

**find(node)** <br>
*it will find the root parent (root node) using path compression*
- loop until we find the root parent
- do path compression i.e compress the height of the trees by   updating the parent
- update the node
- return parent of the node 

**union (node1, node2)** <br>
*it will take two nodes and merge the lower rank node (smaller tree) with the higher rank node*
- find the root parent of the nodes
- if their parents are not the same, we can merge the nodes
    - find the max and min of the parents i.e higher rank and lower rank tree
    - add the lower rank node to the higher rank node by updating the parent of the lower rank node and rank of the higher rank node

#### **Merge Accounts Function**
- create a union-find class objective. Pass the total number of accounts to create parent and rank lists
- set email_acc = a hashmap to map each email to the corresponding account index
    - *key = email*
    - *value = account index*
- set acc_emails = hashmap to map each account index to the list of corresponding emails 
    - *key = account index*
    - *value = list of emails*
- iterate over each account and its emails to merge accounts <br>
for each account index, (account name, list of emails) in the accounts list
    - for each email
        - if the email is already in the email_acc hashmap, merge the current account and the account that contains the same email
        - else add the email and current account index to the email_acc hashmap
- iterate over each email and account index in the email_acc hashmap to find the root account and its emails <br>
for each email and account index in the email_acc hashmap
    - find the root account and add the email to the list of emails of the root account in acc_emails hashmap
- iterate over each account index and its emails in the acc_emails hashmap to prepare the result list <br>
for each account index and its list of emails in the acc_emails 
    - create a sub-list containing the account name and its emails in sorted order and add it to the result list 
- return result list


## Complexity
- Time complexity: (accounts traversal for merging + hashmap traversal for finding root account + result list creation) → O(accounts * account emails * union-find + unique emails * find + accounts * sorting emails) → O(n * k * α(n) + m * α(n) + n * klogk) → O(n * k * constt + m * constt + n * klogk) → O(nk + m + n * klogk) → O(n * klogk)
    - *n = total number of accounts*
    - *m = total number of unique emails*
    - *k = emails belonging to a single account*
    - *α(n) is the inverse Ackermann function, which grows very steadily i.e nearly constant*
    - *the result is a nested list, each sublist contains an account name and its emails*
<!-- Add your time complexity here, e.g. $$O(n)$$ -->


- Space complexity: O(parent list + rank list + email_acc hashmap + acc_emails hashmap + result list) → O(n + n + unique emails + accounts + accounts * account emails) → O(n + n + m + n + n * k) → O(n * k)
<!-- Add your space complexity here, e.g. $$O(n)$$ -->


## Code

In [ ]:
class UnionFind:

    def __init__(self, n):
        self.parent = [i for i in range(n)]
        self.rank = [1] * n
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return self.parent[x]
    
    def union(self, x1, x2):
        p1, p2 = self.find(x1), self.find(x2)
        if p1 != p2:
            maxi = max(p1, p2)
            mini = min(p1, p2)
            self.parent[mini] = maxi
            self.rank[maxi] = mini


class Solution:

    def accountsMerge(self, accounts: List[List[str]]) -> List[List[str]]:
        uf = UnionFind(len(accounts))
        email_acc = {}
        acc_emails = defaultdict(list)
        for i, (_, *emails) in enumerate(accounts):
            for e in emails:
                if e in email_acc:
                    uf.union(i, email_acc[e])
                else:
                    email_acc[e] = i
        for e, a in email_acc.items():
            acc_emails[uf.find(a)].append(e)
        return [[accounts[a][0]] + sorted(emails) for a, emails in acc_emails.items()]